# Deflection Profile Parametric Study for El Ghadioui2020
El Ghadioui2020_Structural behaviour of CFRP reinforced concrete members under bending and shear loads

In [1]:
# %matplotlib widget
%matplotlib qt

In [2]:
from bmcs_cross_section.api import MKappa, ReinforcementType, Reinforcement, TShape, EC2, ConcreteMaterialLaw
from bmcs_beam.api import BoundaryConditions, BoundaryConfig, DeflectionProfile, BeamDesign, LoadDeflectionParamsStudy
from os.path import join
import numpy as np
import pandas as pd
import sympy as sp
import matplotlib.pyplot as plt
import bmcs_utils.api as bu

In [3]:
# Beam B-M-C-K1 - CFRP Bars

excel_file = join('./verification_data/el_ghadioui2020', 'fig_3.xlsx')
data = pd.read_excel(excel_file, engine='openpyxl')
ver_data = [data['B-M-C-K1_deflection'], data['B-M-C-K1_load']]

In [4]:
# Beam B-M-C-K1 - CFRP Bars

# C 50/60

E = 33525

mc = MKappa(reinforcement_type= ReinforcementType.CARBON,
            
            low_kappa = -0.00011,
            high_kappa = 0.00001,
            E_cc = E,
            E_ct = E,
            eps_cr = 0.00006,
            eps_tu = 0.00008,
            eps_cy = 0.00175, # EC2 eps_c3
            eps_cu = 0.0035,  # EC2 eps_cu3
            mu = 0.0)

mc.cross_section_shape.B = 400
mc.cross_section_shape.H = 200
mc.cross_section_layout.reinforcement = [Reinforcement(eps_sy=3600/230000, E=230000, z=200-35, A=4 * 70.14)]

dp = DeflectionProfile(mc=mc)

dp.beam_design.beam_conf_name = BoundaryConfig.CANTILEVER_DIST_LOAD
dp.beam_design.L = 1000

# bu.InteractiveWindow([dp, 
#                       dp.beam_design, 
#                       dp.mc, 
#                       dp.mc.cs_design.cross_section_shape, 
#                       dp.mc.cs_design.cross_section_layout], tight_layout=True).interact()

In [5]:
def run_parametric_study(mcs):
    dp = DeflectionProfile(mc=mc)
    dp.beam_design.beam_conf_name = BoundaryConfig.CANTILEVER_DIST_LOAD
    dp.beam_design.L = 1000
    params_config = [
        ['mc',   dp, mcs],
    ]

    LoadDeflectionParamsStudy(dp).run(params_config)

In [74]:
# Test f_ck
""" [
    {'f_ck': 30, 'low_kappa':0 , 'high_kappa':0.00012 },
    {'f_ck': 50, 'low_kappa':0 , 'high_kappa':0.00012 },
    {'f_ck': 70, 'low_kappa':-0.00001 , 'high_kappa':0.00011},
    {'f_ck': 90, 'low_kappa':-0.00002 , 'high_kappa':0.00011},
    ] """

f_ck = 90

E = EC2.get_E_cm(f_ck)

fctk_0_05 = EC2.get_f_ctk_0_05(f_ck)
f_ctd = fctk_0_05
# f_ctd = 0.85 * fctk_0_05 / 1.5
f_ctd_fl = max((1.6 - 200 /1000) * f_ctd, f_ctd)


mc1 = MKappa(reinforcement_type= ReinforcementType.CARBON,
            low_kappa = -0.00011,
            high_kappa = 0.00001,
            E_cc = E,
            E_ct = E,
            eps_cr = f_ctd_fl/E,            # 0.00006,
            eps_tu = 2* f_ctd_fl/E,         # 0.00008,
            eps_cy = EC2.get_eps_c3(f_ck),
            eps_cu = EC2.get_eps_cu3(f_ck),
            mu = 0.0)

mc1.cross_section_shape.B = 400
mc1.cross_section_shape.H = 200
mc.cross_section_layout.reinforcement = [Reinforcement(eps_sy=3600/230000, E=230000, z=200-35, A=4 * 70.14)]
# mc1.interact()


dp = DeflectionProfile(mc=mc)

dp.beam_design.beam_conf_name = BoundaryConfig.CANTILEVER_DIST_LOAD
dp.beam_design.L = 1000

bu.InteractiveWindow([dp, 
                      dp.beam_design, 
                      dp.mc, 
                      dp.mc.cs_design.cross_section_shape, 
                      dp.mc.cs_design.cross_section_layout], tight_layout=True).interact()

# _, ax = plt.subplots()
# f, w = dp.get_Fw()
# ax.plot(w, f*2/1000)
# ax.plot(ver_data[0], ver_data[1])

No solution The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
No solution The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
M inverse has not succeeded, the M-Kappa solution may have failed due to a wrong kappa range or not suitable material law!
No solution The iteration is not making good progress, as measured by the 
  improvement from the last five Jacobian evaluations.
M inverse has not succeeded, the M-Kappa solution may have failed due to a wrong kappa range or not suitable material law!


In [5]:
# saving mcs with different f_ck

f_ck_setups = [
    {'f_ck': 30, 'low_kappa':0 , 'high_kappa':0.00012 },
    {'f_ck': 50, 'low_kappa':0 , 'high_kappa':0.00012 },
    {'f_ck': 70, 'low_kappa':-0.00001 , 'high_kappa':0.00011},
    {'f_ck': 90, 'low_kappa':-0.00002 , 'high_kappa':0.00011},
    ]
mcs = []
for f_ck_setup in f_ck_setups:
    f_ck = f_ck_setup['f_ck']
    
    E = EC2.get_E_cm(f_ck)
    
    fctk_0_05 = EC2.get_f_ctk_0_05(f_ck)
    f_ctd = fctk_0_05
    # f_ctd = 0.85 * fctk_0_05 / 1.5
    f_ctd_fl = max((1.6 - 200 /1000) * f_ctd, f_ctd)

    mc1 = MKappa(reinforcement_type= ReinforcementType.CARBON,
                low_kappa = f_ck_setup['low_kappa'],
                high_kappa = f_ck_setup['high_kappa'],
                E_cc = E,
                E_ct = E,
                eps_cr = f_ctd_fl/E,            # 0.00006,
                eps_tu = 2* f_ctd_fl/E,         # 0.00008,
                eps_cy = EC2.get_eps_c3(f_ck),
                eps_cu = EC2.get_eps_cu3(f_ck), # EC2 eps_cu3
                mu = 0.0)
    mc1.cross_section_shape.B = 400
    mc1.cross_section_shape.H = 200
    mc1.cross_section_layout.reinforcement = [Reinforcement(eps_sy=0.014, E=135121, z=35, A=2 * 70.14)]
    mcs.append(mc1)
    

In [26]:
# Test E

""" [
    {'E_c': 30000, 'low_kappa':-0.00011 , 'high_kappa':0.00001 },
    {'E_c': 35000, 'low_kappa':-0.00011 , 'high_kappa':0.00001 },
    {'E_c': 40000, 'low_kappa':-0.00011 , 'high_kappa':0.00001 },
    {'E_c': 45000, 'low_kappa':-0.00011 , 'high_kappa':0.00001 },
    ] """


E = 30000

mc1 = MKappa(reinforcement_type= ReinforcementType.CARBON,           
            low_kappa = -0.00011,
            high_kappa = 0.00001,
            E_cc = E,
            E_ct = E,
            eps_cr = 0.00006,
            eps_tu = 0.00008,
            eps_cy = 0.00175,
            eps_cu = 0.0035,
            mu = 0.0)

mc1.cross_section_shape.B = 400
mc1.cross_section_shape.H = 200
mc.cross_section_layout.reinforcement = [Reinforcement(eps_sy=3600/230000, E=230000, z=200-35, A=4 * 70.14)]
# mc1.interact()


dp = DeflectionProfile(mc=mc)

dp.beam_design.beam_conf_name = BoundaryConfig.CANTILEVER_DIST_LOAD
dp.beam_design.L = 1000

bu.InteractiveWindow([dp, 
                      dp.beam_design, 
                      dp.mc, 
                      dp.mc.cs_design.cross_section_shape, 
                      dp.mc.cs_design.cross_section_layout], tight_layout=True).interact()

# _, ax = plt.subplots()
# f, w = dp.get_Fw()
# ax.plot(w, f*2/1000)
# ax.plot(ver_data[0], ver_data[1])

In [6]:
# saving mcs with different E

setups = [
    {'E_c': 30000, 'low_kappa':-0.00011 , 'high_kappa':0.00001 },
    {'E_c': 35000, 'low_kappa':-0.00011 , 'high_kappa':0.00001 },
    {'E_c': 40000, 'low_kappa':-0.00011 , 'high_kappa':0.00001 },
    {'E_c': 45000, 'low_kappa':-0.00011 , 'high_kappa':0.00001 },
    ]
mcs = []
for setup in setups:
    
    E = setup['E_c']
    
    mc1 = MKappa(reinforcement_type= ReinforcementType.CARBON,
                low_kappa = setup['low_kappa'],
                high_kappa = setup['high_kappa'],
                E_cc = E,
                E_ct = E,
                eps_cr = 0.00006,
                eps_tu = 0.00008,
                eps_cy = 0.00175,
                eps_cu = 0.0035,
                mu = 0.0)
    mc1.cross_section_shape.B = 400
    mc1.cross_section_shape.H = 200
    mc1.cross_section_layout.reinforcement = [Reinforcement(eps_sy=3600/230000, E=230000, z=200-35, A=4 * 70.14)]
    mcs.append(mc1)
    

In [7]:
# running parametric study for different E
run_parametric_study(mcs)

Parametric study is running...
mc: <bmcs_cross_section.mkappa.mkappa.MKappa object at 0x000002191B122180>, <bmcs_cross_section.mkappa.mkappa.MKappa object at 0x000002191CDC0770>, <bmcs_cross_section.mkappa.mkappa.MKappa object at 0x000002191CD984A0>, <bmcs_cross_section.mkappa.mkappa.MKappa object at 0x000002191D129680>, 
Parametric study finished.


In [31]:
# Test E_s

E = 33525

mc1 = MKappa(reinforcement_type= ReinforcementType.CARBON,           
            low_kappa = -0.00011,
            high_kappa = 0.00001,
            E_cc = E,
            E_ct = E,
            eps_cr = 0.00006,
            eps_tu = 0.00008,
            eps_cy = 0.00175,
            eps_cu = 0.0035,
            mu = 0.0)

mc1.cross_section_shape.B = 400
mc1.cross_section_shape.H = 200
mc1.cross_section_layout.reinforcement = [Reinforcement(eps_sy=3600/230000, E=230000, z=200-35, A=4 * 70.14)]
# mc1.interact()


dp = DeflectionProfile(mc=mc1)

dp.beam_design.beam_conf_name = BoundaryConfig.CANTILEVER_DIST_LOAD
dp.beam_design.L = 1000

bu.InteractiveWindow([dp, 
                      dp.beam_design, 
                      dp.mc, 
                      dp.mc.cs_design.cross_section_shape, 
                      dp.mc.cs_design.cross_section_layout], tight_layout=True).interact()

# _, ax = plt.subplots()
# f, w = dp.get_Fw()
# ax.plot(w, f*2/1000)
# ax.plot(ver_data[0], ver_data[1])

In [8]:
# saving mcs with different E_s

setups = [
    {'E_s': 130000, 'low_kappa':-0.00011 , 'high_kappa':0.00001 },
    {'E_s': 170000, 'low_kappa':-0.00011 , 'high_kappa':0.00001 },
    {'E_s': 210000, 'low_kappa':-0.00011 , 'high_kappa':0.00001 },
    {'E_s': 250000, 'low_kappa':-0.00011 , 'high_kappa':0.00001 },
    ]
mcs = []
for setup in setups:
    
    E = 33525
    
    mc1 = MKappa(reinforcement_type= ReinforcementType.CARBON,
                low_kappa = setup['low_kappa'],
                high_kappa = setup['high_kappa'],
                E_cc = E,
                E_ct = E,
                eps_cr = 0.00006,
                eps_tu = 0.00008,
                eps_cy = 0.00175,
                eps_cu = 0.0035,
                mu = 0.0)
    mc1.cross_section_shape.B = 400
    mc1.cross_section_shape.H = 200
    mc1.cross_section_layout.reinforcement = [Reinforcement(eps_sy=3600/230000, E=setup['E_s'], z=200-35, A=4 * 70.14)]
    mcs.append(mc1)
    

In [9]:
# running parametric study for different E_s
run_parametric_study(mcs)

Parametric study is running...
mc: <bmcs_cross_section.mkappa.mkappa.MKappa object at 0x000002191CF65DB0>, <bmcs_cross_section.mkappa.mkappa.MKappa object at 0x000002191D98C900>, <bmcs_cross_section.mkappa.mkappa.MKappa object at 0x000002191D4644A0>, <bmcs_cross_section.mkappa.mkappa.MKappa object at 0x000002191DA95270>, 
Parametric study finished.


In [38]:
# Test carbon strength

""" [
    {'eps_sy': 1800/230000, 'low_kappa':-0.00005 , 'high_kappa':0.00001 },
    {'eps_sy': 2200/230000, 'low_kappa':-0.00007 , 'high_kappa':0.00001 },
    {'eps_sy': 2600/230000, 'low_kappa':-0.00008 , 'high_kappa':0.00001 },
    {'eps_sy': 3000/230000, 'low_kappa':-0.00010 , 'high_kappa':0.00001 },
    ] """

E = 33525

mc1 = MKappa(reinforcement_type= ReinforcementType.CARBON,           
            low_kappa = -0.00010,
            high_kappa = 0.00001,
            E_cc = E,
            E_ct = E,
            eps_cr = 0.00006,
            eps_tu = 0.00008,
            eps_cy = 0.00175,
            eps_cu = 0.0035,
            mu = 0.0)

mc1.cross_section_shape.B = 400
mc1.cross_section_shape.H = 200
mc1.cross_section_layout.reinforcement = [Reinforcement(eps_sy=3000/230000, E=230000, z=200-35, A=4 * 70.14)]
# mc1.interact()


dp = DeflectionProfile(mc=mc1)

dp.beam_design.beam_conf_name = BoundaryConfig.CANTILEVER_DIST_LOAD
dp.beam_design.L = 1000

bu.InteractiveWindow([dp, 
                      dp.beam_design, 
                      dp.mc, 
                      dp.mc.cs_design.cross_section_shape, 
                      dp.mc.cs_design.cross_section_layout], tight_layout=True).interact()

In [10]:
# saving mcs with different carbon strength

setups = [
    {'eps_sy': 1800/230000, 'low_kappa':-0.00005 , 'high_kappa':0.00001 },
    {'eps_sy': 2200/230000, 'low_kappa':-0.00007 , 'high_kappa':0.00001 },
    {'eps_sy': 2600/230000, 'low_kappa':-0.00008 , 'high_kappa':0.00001 },
    {'eps_sy': 3000/230000, 'low_kappa':-0.00010 , 'high_kappa':0.00001 },
    ]
mcs = []
for setup in setups:
    
    E = 33525
    
    mc1 = MKappa(reinforcement_type= ReinforcementType.CARBON,
                low_kappa = setup['low_kappa'],
                high_kappa = setup['high_kappa'],
                E_cc = E,
                E_ct = E,
                eps_cr = 0.00006,
                eps_tu = 0.00008,
                eps_cy = 0.00175,
                eps_cu = 0.0035,
                mu = 0.0)
    mc1.cross_section_shape.B = 400
    mc1.cross_section_shape.H = 200
    mc1.cross_section_layout.reinforcement = [Reinforcement(eps_sy=setup['eps_sy'], E=230000, z=200-35, A=4 * 70.14)]
    mcs.append(mc1)
    

In [11]:
# running parametric study for different carbon strength
run_parametric_study(mcs)

Parametric study is running...
mc: <bmcs_cross_section.mkappa.mkappa.MKappa object at 0x000002191CF65DB0>, <bmcs_cross_section.mkappa.mkappa.MKappa object at 0x000002191FC863B0>, <bmcs_cross_section.mkappa.mkappa.MKappa object at 0x000002191FE26680>, <bmcs_cross_section.mkappa.mkappa.MKappa object at 0x000002191D603F90>, 
Parametric study finished.


In [56]:
# Test concrete compressive strength

""" [
    {'concrete_material_law': ConcreteMaterialLaw.EC2, 'f_cd': 30, 'low_kappa':-0.00008 , 'high_kappa':0.00001 },
    {'concrete_material_law': ConcreteMaterialLaw.EC2, 'f_cd': 50, 'low_kappa':-0.00009 , 'high_kappa':0.00001 },
    {'concrete_material_law': ConcreteMaterialLaw.EC2, 'f_cd': 70, 'low_kappa':-0.0001 , 'high_kappa':0.00001 },
    {'concrete_material_law': ConcreteMaterialLaw.EC2, 'f_cd': 90, 'low_kappa':-0.00012 , 'high_kappa':0.00001 },
    ] """

E = 33525

mc1 = MKappa(reinforcement_type= ReinforcementType.CARBON,   
             
            concrete_material_law = ConcreteMaterialLaw.EC2,
            f_cd = 70, 
             
            low_kappa = -0.00012,
            high_kappa = 0.00001,
            E_cc = E,
            E_ct = E,
            eps_cr = 0.00006,
            eps_tu = 0.00008,
            eps_cy = 0.00175,
            eps_cu = 0.0035,
            mu = 0.0)

mc1.cross_section_shape.B = 400
mc1.cross_section_shape.H = 200
mc1.cross_section_layout.reinforcement = [Reinforcement(eps_sy=3600/230000, E=230000, z=200-35, A=4 * 70.14)]

dp = DeflectionProfile(mc=mc1)

dp.beam_design.beam_conf_name = BoundaryConfig.CANTILEVER_DIST_LOAD
dp.beam_design.L = 1000

bu.InteractiveWindow([dp, 
                      dp.beam_design, 
                      dp.mc, 
                      dp.mc.cs_design.cross_section_shape, 
                      dp.mc.cs_design.cross_section_layout], tight_layout=True).interact()

In [12]:
# saving mcs with different concrete compressive strength

setups = [
    {'concrete_material_law': ConcreteMaterialLaw.EC2, 'f_cd': 30, 'low_kappa':-0.00008 , 'high_kappa':0.00001 },
    {'concrete_material_law': ConcreteMaterialLaw.EC2, 'f_cd': 50, 'low_kappa':-0.00009 , 'high_kappa':0.00001 },
    {'concrete_material_law': ConcreteMaterialLaw.EC2, 'f_cd': 70, 'low_kappa':-0.0001 , 'high_kappa':0.00001 },
    {'concrete_material_law': ConcreteMaterialLaw.EC2, 'f_cd': 90, 'low_kappa':-0.00012 , 'high_kappa':0.00001 },
    ]
mcs = []
for setup in setups:
    
    E = 33525
    
    mc1 = MKappa(reinforcement_type= ReinforcementType.CARBON,
                low_kappa = setup['low_kappa'],
                high_kappa = setup['high_kappa'],

                 concrete_material_law = setup['concrete_material_law'],
                 f_cd = setup['f_cd'],
                 
                E_cc = E,
                E_ct = E,
                eps_cr = 0.00006,
                eps_tu = 0.00008,
                eps_cy = 0.00175,
                eps_cu = 0.0035,
                mu = 0.0)
    mc1.cross_section_shape.B = 400
    mc1.cross_section_shape.H = 200
    mc1.cross_section_layout.reinforcement = [Reinforcement(eps_sy=3600/230000, E=230000, z=200-35, A=4 * 70.14)]
    mcs.append(mc1)
    

In [13]:
# running parametric study for different concrete compressive strength
run_parametric_study(mcs)

Parametric study is running...
mc: <bmcs_cross_section.mkappa.mkappa.MKappa object at 0x000002191CF65DB0>, <bmcs_cross_section.mkappa.mkappa.MKappa object at 0x0000021920514630>, <bmcs_cross_section.mkappa.mkappa.MKappa object at 0x000002191FD1D7C0>, <bmcs_cross_section.mkappa.mkappa.MKappa object at 0x0000021920498630>, 
Parametric study finished.


In [62]:
# Test concrete tensile strength

""" [
    {'f_ck': 30, 'low_kappa':-0.00009 , 'high_kappa':0.00001 },
    {'f_ck': 50, 'low_kappa':-0.00009 , 'high_kappa':0.00001 },
    {'f_ck': 70, 'low_kappa':-0.00009 , 'high_kappa':0.00001 },
    {'f_ck': 90, 'low_kappa':-0.00009 , 'high_kappa':0.00001 },
    ] """

E = 33525

f_ck = 30

fctk_0_05 = EC2.get_f_ctk_0_05(f_ck)
f_ctd = fctk_0_05
# f_ctd = 0.85 * fctk_0_05 / 1.5
f_ctd_fl = max((1.6 - 200 /1000) * f_ctd, f_ctd)

mc1 = MKappa(reinforcement_type= ReinforcementType.CARBON,  
             
             n_kappa = 500,
                         
            low_kappa = -0.00009,
            high_kappa = 0.00001,
            E_cc = E,
            E_ct = E,
            eps_cr = f_ctd_fl/E,            # 0.00006,
            eps_tu = 2* f_ctd_fl/E,         # 0.00008,
            eps_cy = 0.00175,
            eps_cu = 0.0035,
            mu = 0.0)

mc1.cross_section_shape.B = 400
mc1.cross_section_shape.H = 200
mc1.cross_section_layout.reinforcement = [Reinforcement(eps_sy=3600/230000, E=230000, z=200-35, A=4 * 70.14)]

dp = DeflectionProfile(mc=mc1)

dp.beam_design.beam_conf_name = BoundaryConfig.CANTILEVER_DIST_LOAD
dp.beam_design.L = 1000

bu.InteractiveWindow([dp, 
                      dp.beam_design, 
                      dp.mc, 
                      dp.mc.cs_design.cross_section_shape, 
                      dp.mc.cs_design.cross_section_layout], tight_layout=True).interact()

In [14]:
# saving mcs with different concrete tensile strength

setups = [
    {'f_ck': 30, 'low_kappa':-0.00009 , 'high_kappa':0.00001 },
    {'f_ck': 50, 'low_kappa':-0.00009 , 'high_kappa':0.00001 },
    {'f_ck': 70, 'low_kappa':-0.00009 , 'high_kappa':0.00001 },
    {'f_ck': 90, 'low_kappa':-0.00009 , 'high_kappa':0.00001 },
    ]
mcs = []
for setup in setups:
    
    E = 33525
    
    f_ck = setup['f_ck']

    fctk_0_05 = EC2.get_f_ctk_0_05(f_ck)
    f_ctd = fctk_0_05
    # f_ctd = 0.85 * fctk_0_05 / 1.5
    f_ctd_fl = max((1.6 - 200 /1000) * f_ctd, f_ctd)
    
    mc1 = MKappa(reinforcement_type= ReinforcementType.CARBON,
                low_kappa = setup['low_kappa'],
                high_kappa = setup['high_kappa'],         
                 n_kappa = 400,
                E_cc = E,
                E_ct = E,
                eps_cr = f_ctd_fl/E,            # 0.00006,
                eps_tu = 2* f_ctd_fl/E,         # 0.00008,
                eps_cy = 0.00175,
                eps_cu = 0.0035,
                mu = 0.0)
    mc1.cross_section_shape.B = 400
    mc1.cross_section_shape.H = 200
    mc1.cross_section_layout.reinforcement = [Reinforcement(eps_sy=3600/230000, E=230000, z=200-35, A=4 * 70.14)]
    mcs.append(mc1)
    

In [15]:
# running parametric study for different concrete tensile strength
# run_parametric_study(mcs)

dp = DeflectionProfile(mc=mc, n_load_steps=200)
dp.beam_design.beam_conf_name = BoundaryConfig.CANTILEVER_DIST_LOAD
dp.beam_design.L = 1000
params_config = [
    ['mc',   dp, mcs],
]

LoadDeflectionParamsStudy(dp).run(params_config)

Parametric study is running...
mc: <bmcs_cross_section.mkappa.mkappa.MKappa object at 0x000002191D794BD0>, <bmcs_cross_section.mkappa.mkappa.MKappa object at 0x0000021920802040>, <bmcs_cross_section.mkappa.mkappa.MKappa object at 0x0000021920930720>, <bmcs_cross_section.mkappa.mkappa.MKappa object at 0x00000219203F0090>, 
Parametric study finished.


In [6]:
dp = DeflectionProfile(mc=mc)

dp.beam_design.beam_conf_name = BoundaryConfig.CANTILEVER_DIST_LOAD
dp.beam_design.L = 1000

params_config = [
    ['mc',      dp, mcs],
#         ['L',      dp.beam_design, [5000]],
#         ['B',      mc.cross_section_shape, [300, 500]],
#         ['H',      mc.cross_section_shape, [21505, 300]],
#         ['E_ct',   mc, [33525, 34000]],
#         ['E_cc',   mc, [33525, 34000]],
#         ['eps_cr', mc, [0.00003, 0.00005, 0.00006]],
#         ['eps_cy', mc, [0.0015, 0.00175, 0.002, 0.0025]],
#         ['eps_cu', mc, [0.002, 0.0025, 0.0035, 0.004]],
#         ['eps_tu', mc, [0.00004, 0.00005, 0.00006]],
#         ['mu',     mc, [0, 1]],
#         ['z',      mc.cross_section_layout.reinforcement[0], [1, 29]],
#         ['A',      mc.cross_section_layout.reinforcement[0], [70.14, 2* 70.14]],
#         ['E',      mc.cross_section_layout.reinforcement[0], [130000, 135121, 140000]],
#         ['eps_sy', mc.cross_section_layout.reinforcement[0], [0.010, 0.014, 0.018]],
#         ['n_x',    dp.beam_design, [10, 1000]],
]

LoadDeflectionParamsStudy(dp).run(params_config, ver_data)

Parametric study is running...
mc: <bmcs_cross_section.mkappa.mkappa.MKappa object at 0x0000024AA11DCF90>, <bmcs_cross_section.mkappa.mkappa.MKappa object at 0x0000024AA1251860>, <bmcs_cross_section.mkappa.mkappa.MKappa object at 0x0000024AA12FD630>, <bmcs_cross_section.mkappa.mkappa.MKappa object at 0x0000024AA12FD9F0>, 
Parametric study finished.
